In [29]:
%%time 
from data import *
from data.pair_feature import *

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 60.6 µs


In [2]:
simple_features_train.shape

(2991396, 9)

In [3]:
simple_features_test.shape

(1044196, 9)

In [4]:
from xgboost.sklearn import XGBClassifier

In [5]:
from sklearn.cross_validation import train_test_split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(simple_features_train.values, item_pairs_train.isDuplicate.values, test_size=0.2)

In [13]:
model

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [18]:
model = XGBClassifier(learning_rate=0.1, max_depth=5, subsample=0.8, colsample_bytree=0.8, n_estimators=260)

In [20]:
model.fit(X_train, y_train, eval_metric='auc', eval_set=[(X_test, y_test)], early_stopping_rounds=20)

[0]	validation_0-auc:0.774673
Will train until validation_0-auc hasn't improved in 20 rounds.
[1]	validation_0-auc:0.775424
[2]	validation_0-auc:0.77756
[3]	validation_0-auc:0.778537
[4]	validation_0-auc:0.778272
[5]	validation_0-auc:0.779497
[6]	validation_0-auc:0.782346
[7]	validation_0-auc:0.782457
[8]	validation_0-auc:0.782174
[9]	validation_0-auc:0.782954
[10]	validation_0-auc:0.783063
[11]	validation_0-auc:0.783381
[12]	validation_0-auc:0.783512
[13]	validation_0-auc:0.783989
[14]	validation_0-auc:0.784288
[15]	validation_0-auc:0.784368
[16]	validation_0-auc:0.784787
[17]	validation_0-auc:0.784884
[18]	validation_0-auc:0.785146
[19]	validation_0-auc:0.785673
[20]	validation_0-auc:0.786072
[21]	validation_0-auc:0.786126
[22]	validation_0-auc:0.786214
[23]	validation_0-auc:0.787033
[24]	validation_0-auc:0.787448
[25]	validation_0-auc:0.787491
[26]	validation_0-auc:0.787789
[27]	validation_0-auc:0.788035
[28]	validation_0-auc:0.788157
[29]	validation_0-auc:0.788343
[30]	validation_0

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=260, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.8)

In [24]:
pd.DataFrame({'feature': simple_features_train.columns,
              'importance': model.feature_importances_}).sort('importance', ascending=False)

/home/idm/.local/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


,feature,importance
2,price_diff_ratio,0.382829
0,attrsJSON_item_jaccard,0.234728
1,attrsJSON_key_jaccard,0.156443
8,same_title,0.065452
5,same_lat_lon,0.045431
7,same_metroID,0.041453
6,same_locationID,0.040426
4,same_description,0.033239
3,same_categoryID,0.000000


In [22]:
%time model.fit(simple_features_train, item_pairs_train.isDuplicate)

CPU times: user 2h 23min 47s, sys: 4min 40s, total: 2h 28min 28s
Wall time: 3min 16s


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=260, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.8)

In [23]:
model.classes_

array([0, 1])

In [25]:
prob_test = model.predict_proba(simple_features_test)[:,1]

In [26]:
item_pairs_test.head()

,id,itemID_1,itemID_2
0,0,5,4670875
1,1,5,787210
2,2,6,1705280
3,3,11,3020777
4,4,23,5316130


In [28]:
pd.DataFrame({'id': item_pairs_test['id'], 'probability': prob_test}).to_csv('baseline.csv', index=False)